In [1]:
from vivarium_inputs.interface import get_measure
import gbd_mapping as gbd
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import vivarium_gbd_access
from operator import attrgetter
from get_draws.api import get_draws
%load_ext autoreload
%autoreload 2

!date

Mon Mar 23 17:07:30 PDT 2020


- measure_id for prevalence is 5
- gbd_round_id is 5
- rei_id for Vitamin A deficiency is 96
- modelable_entity_id for Vitamin A deficiency is 2510
- age_group_id for early neonatal is 2, for late neonatal is 3, for post neonatal is 4, for year 1 to 4 is 5
- location_id for Nigeria is 214, for India is 163, for Ethiopia is 179
- year_id is 2017

### 1. pull prevalence (from the VAD cause)

In [2]:
prev_df = pd.DataFrame()
for location in ['India','Nigeria','Ethiopia']:
    data = get_measure(gbd.causes.vitamin_a_deficiency, "prevalence", location).reset_index()
    data['year'] = data['year'].map(attrgetter('left'))
    data['age_start'] = data['age'].map(attrgetter('left'))
    data['age_end'] = data['age'].map(attrgetter('right'))
    #keep under 5 year old and year 2017
    data = data[(data['age_end'] == 5) & (data['year'] == 2017)]
    prev_df = prev_df.append(data)

/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:260: UserWarning: Sequela prevalence data for cause vitamin_a_deficiency may not correctly aggregate up to the cause level in all locations. Sequela models may not be consistent with models for this cause.
  warnings.warn(f"{children.capitalize()} {measure} data for cause {entity.name} may not correctly "
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:260: UserWarning: Sequela prevalence data for cause vitamin_a_deficiency may not correctly aggregate up to the cause level in all locations. Sequela models may not be consistent with models for this cause.
  warnings.warn(f"{children.capitalize()} {measure} data for cause {entity.name} may not correctly "
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:260: UserWarning: Sequela prevalence data

In [3]:
#data source: como using cause id 389, age 1-5
df_prev = prev_df.melt(id_vars = ['location', 'sex', 'age','year','age_start','age_end'], 
                      var_name = 'draw_number', value_name = 'value')
df_prev.groupby('location')['value'].describe(percentiles=[.025, .975])[['mean','2.5%','97.5%']]

,mean,2.5%,97.5%
location,,,
Ethiopia,0.315128,0.252618,0.379726
India,0.250471,0.205456,0.303530
Nigeria,0.232223,0.184475,0.288078


In [4]:
def pullData(measure, measure_id):
    vitA_def = get_draws("rei_id", 96, location_id=[214, 163, 179], age_group_id=[5],
                     year_id=2017, sex_id=[1,2], source="exposure", gbd_round_id=5)
    df = vitA_def.query(f'measure_id == {measure_id} and modelable_entity_id == 2510')
    df_final = df.melt(id_vars = ['rei_id', 'modelable_entity_id', 'location_id', 'year_id','age_group_id', 'sex_id', 'parameter', 'measure_id', 'metric_id'], 
                      var_name = 'draw_number', value_name = 'value')
    df_final['location_id'].replace({163:'India', 179:'Ethiopia', 214:'Nigeria'}, inplace = True)
    print(f'{measure} summary statistics (mean, upper, lower) for each of the under 5 age groups:')
    print(df_final.groupby(['location_id'])['value'].describe(percentiles=[.025, .975])[['mean','2.5%','97.5%']])
    print('\n')

In [5]:
#exposure source using rei_id 96, age 1-5
pullData('Prevalence', 5) 

Prevalence summary statistics (mean, upper, lower) for each of the under 5 age groups:
                 mean      2.5%     97.5%
location_id                              
Ethiopia     0.314543  0.252409  0.379091
India        0.249882  0.205045  0.302771
Nigeria      0.231538  0.183927  0.287513




### 2. pull relative risks for measles, diarrhea, LRI from the VAD risk
(GBD adjusted the relative risk for the background prevalence of
Vitamin A deficiency in 1-4 years from the GBD 2017 model )

In [6]:
rr_df = pd.DataFrame()
for location in ['India','Nigeria','Ethiopia']:
    data = get_measure(gbd.risk_factors.vitamin_a_deficiency, "relative_risk", location).reset_index()
    data['year'] = data['year'].map(attrgetter('left'))
    data['age_start'] = data['age'].map(attrgetter('left'))
    data['age_end'] = data['age'].map(attrgetter('right'))
    #keep under 5 year old, year 2017, parameter cat1
    data = data[(data['age_end'] == 5) & (data['year'] == 2017) & (data['parameter'] == 'cat1')]
    rr_df = rr_df.append(data)

/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:1384: UserWarning: Relative_risk for risk_factor vitamin_a_deficiency may be outside the normal range.
  warnings.warn(f'{measure.capitalize()} for {entity.kind} {entity.name} may be outside the normal range.')
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:1405: UserWarning: Risk_factor vitamin_a_deficiency relative_risk data may violate the following restrictions:  age restriction.
  warnings.warn(f'{entity.kind.capitalize()} {entity.name} {measure} data may violate the '
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:1856: UserWarning: Data does not contain all age groups in restriction range.
  warnings.warn('Data does not contain all age groups in restriction range.')
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/

In [7]:
##RR for under 5 year old
df_rr = rr_df.melt(id_vars = ['location', 'sex', 'age', 'year', 'affected_entity', 'affected_measure',
                                'parameter', 'age_start', 'age_end'], var_name = 'draw_number', value_name = 'value')
df_rr.groupby(['location','affected_entity'])['value'].describe(percentiles=[.025, .975])[['mean','2.5%','97.5%']]

mean      2.5%     97.5%
location affected_entity                                           
Ethiopia diarrheal_diseases            2.443922  2.268101  2.626098
         lower_respiratory_infections  1.329526  1.113526  1.585032
         measles                       3.514660  2.526005  4.670518
India    diarrheal_diseases            2.443922  2.268101  2.626098
         lower_respiratory_infections  1.329526  1.113526  1.585032
         measles                       3.514660  2.526005  4.670518
Nigeria  diarrheal_diseases            2.443922  2.268101  2.626098
         lower_respiratory_infections  1.329526  1.113526  1.585032
         measles                       3.514660  2.526005  4.670518